In [2]:
import chainer
import chainer.functions as F
import chainer.links as L
import numba
from numba import jit
from chainer import training
from chainer.training import extensions
from chainer.datasets import TupleDataset
from chainer.dataset import DatasetMixin
import pandas as pd 
import PIL 
from PIL import Image 
import numpy as np
from chainer.dataset import concat_examples
from chainer.cuda import to_cpu

from chainer import cuda, Chain
from chainer import optimizers
from chainer import serializers
from chainer import links, functions, Variable

from scipy import ndimage, misc


/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def source_image_data(data_file, label_col, image_col, remove_nas): 
    
    if remove_nas:
        data_file = data_file.dropna(subset=[image_col, label_col])  # remove missing data for now 
    
    image_list = data_file[image_col].tolist()
    label_list = data_file[label_col].tolist()
    
    return image_list, label_list 


In [4]:
def convert_to_greyscale(fp, new_path, imname):
    """Converts an image to greyscale"""
    img = Image.open(fp).convert('L')
    np = new_path + imname
    img.save(np)
    return np

In [5]:
def resize_iamge(fp):
    
    image = ndimage.imread(fp, mode="L")
    image_resized = misc.imresize(image, (112, 112))
    return image_resized

def get_image(image_path):
    """Get a numpy array of an image so that one can access values[x][y]."""
    image = Image.open(image_path, 'r')
    print(image.size)
    width, height = image.size
    pixel_values = list(image.getdata())
    if image.mode == 'RGB':
        channels = 3
    elif image.mode == 'L':
        channels = 1
    else:
        print("Unknown mode: %s" % image.mode)
        return None
    pixel_values = np.array(pixel_values).reshape(width, height, channels)
    return pixel_values

def load_images(path_name, image_list): 
    
    imlist = []
    for i in image_list:
        #fp = path_name + i
        fp = convert_to_greyscale(path_name + i, "images/", "grey_" + i)
        image = ndimage.imread(fp, mode = "L")
        image_resized = misc.imresize(image, (128, 128, 3))
#         image_resized = np.expand_dims(image_resized, axis=0)
        prep_image = chainer.links.model.vision.vgg.prepare(image_resized)
        imlist.append(prep_image)
        
    return imlist

In [6]:
def format_for_chainer(images, labels): 

    return TupleDataset(images, labels)

In [7]:
gpu_id = -1

# hardcoded garbage that gets the csv and the appropriate labels/image data 
data_file = pd.read_csv("/home/ccaggian/bachelor_data/bachelor_females_images.csv") 
image_folder_path = "/home/ccaggian/bachelor_data/images/"

label_col = "ElimWeek"  # column with labels (week eliminated)
image_col = "image"  # column with data (jpegs)

images, labels = source_image_data(data_file, label_col, image_col, True)  # get appropriate data in list for chainer, remove nans

images = load_images(image_folder_path, images)  # give appropriate absolute path of image data 

train_data = format_for_chainer(images, labels)
train_data

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [280]:
class ConvBlock(chainer.Chain):
    
    def __init__(self, n_ch, pool_drop=False):
        w = chainer.initializers.HeNormal()
        super(ConvBlock, self).__init__()
        with self.init_scope():
            self.conv=L.Convolution2D(None, n_ch, 3, 1, 1, nobias=True, initialW=w)
            self.bn=L.BatchNormalization(n_ch)
        
        self.pool_drop = pool_drop
        
    def __call__(self, x):
        h = F.relu(self.bn(self.conv(x)))
        if self.pool_drop:
            h = F.max_pooling_2d(h, 2, 2)
            h = F.dropout(h, ratio=0.25)
        return h
    
class LinearBlock(chainer.Chain):
    
    def __init__(self):
        w = chainer.initializers.HeNormal()
        super(LinearBlock, self).__init__()
        with self.init_scope():
            self.fc=L.Linear(None, 1024, initialW=w)
        
    def __call__(self, x):
        return F.dropout(F.relu(self.fc(x)), ratio=0.5)

In [281]:
class DeepCNN(chainer.ChainList):

    def __init__(self, n_output):
        super(DeepCNN, self).__init__(
            ConvBlock(64),
            ConvBlock(64, True),
            ConvBlock(128),
            ConvBlock(128, True),
            ConvBlock(256),
            ConvBlock(256),
            ConvBlock(256),
            ConvBlock(256, True),
            LinearBlock(),
            LinearBlock(),
            L.Linear(None, n_output)
        )
        

    def __call__(self, x):
        for f in self.children():
            x = f(x)
        return x

In [9]:
class VGG(Chain):

    def __init__(self):
        super(VGG, self).__init__()
        with self.init_scope():
            self.base = L.VGG16Layers()  


    def __call__(self, x):
        h = self.base (x, layers = ["fc7"])["fc7"]
        return h

In [10]:
#model = L.Classifier(DeepCNN(1))
    
# Setup an optimizer
# optimizer = chainer.optimizers.SGD()
# optimizer.setup(model)
    
# LOAD OUR DATASET HERE INSTEAD OF MNIST
# test_iter = chainer.iterators.SerialIterator(test, batch_size=100, repeat=False, shuffle=False)
    
# Set up a trainer
# updater = training.StandardUpdater(train_iter, optimizer, device=-1)
# trainer = training.Trainer(updater, (5, 'epoch'), out='result')
    
# trainer.extend(extensions.Evaluator(test_iter, model, device=-1))
# trainer.extend(extensions.LogReport())
# trainer.extend(extensions.PrintReport( ['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy']))
#trainer.extend(extensions.ProgressBar())
# trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'], x_key='epoch', file_name='loss.png'))
# trainer.extend(extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'], x_key='epoch', file_name='accuracy.png'))
    
# Run the training
#trainer.run()

In [12]:
model = VGG()

train_iter = chainer.iterators.SerialIterator(train_data, batch_size=1)

labs = []
features = []

train_batch = train_iter.next()
image_train, target_train = concat_examples(train_batch, -1)
prediction_train = model(image_train)

has_next = True
while has_next:
    
    try:
        train_batch = train_iter.next()
        image_train, target_train = concat_example(train_batch, -1)
        proc = model(image_train)
        labs.append(target_train[0])
        



[5.]
